# Open Source Models Evaluation

The aim of this notebook is to evaluate the performance of open source models on test data.

⚠️ *This evaluation in a separate file is intended to avoid problems related to CUDA and GPU memory, which can only be released by restarting the kernel for certain configurations*.

In [1]:
# ---------------------------- PREPARING NOTEBOOK ---------------------------- #
# Autoreload
%load_ext autoreload
%autoreload 2

# Random seed
import numpy as np
np.random.seed(42)

# External modules
import os
from IPython.display import display, Markdown, Latex, clear_output
from tqdm import notebook as tqdm

# Set global log level
import logging
logging.basicConfig(level=logging.INFO)
os.environ['TOKENIZERS_PARALLELISM'] = 'false'

# Define PWD as the current git repository
import git
repo = git.Repo('.', search_parent_directories=True)
pwd = repo.working_dir
os.chdir(pwd)

## BERT

In [2]:
# --------------------------- LOAD AND PREPARE DATA -------------------------- #
import os
import pandas as pd

csv_list = {}
for file in os.listdir(os.path.join(pwd, "data", "raw", "difficulty_estimation")):
    if file.endswith(".csv"):
        file_key = [
            "_".join(os.path.splitext(file)[0].split("_")[:-1]),
            os.path.splitext(file)[0].split("_")[-1],
        ]
        if file_key[0] not in csv_list:
            csv_list[file_key[0]] = {}
        csv_list[file_key[0]][file_key[1]] = pd.read_csv(
            os.path.join(pwd, "data", "raw", "difficulty_estimation", file)
        )

In [ ]:
# ------------------------------ PREPARE DATASET ----------------------------- #
from transformers import AutoTokenizer
from datasets import Dataset
from sklearn.preprocessing import LabelEncoder
import pandas as pd


def prepare_bert_datasets(df: pd.DataFrame, tokenizer: AutoTokenizer):
    # Encode labels
    label_encoder = LabelEncoder()
    df["difficulty"] = label_encoder.fit_transform(df["difficulty"])

    # Create dataset
    dataset = Dataset.from_pandas(df)

    # Tokenize les phrases
    encoded_dataset = dataset.map(
        lambda e: tokenizer(e["sentence"], truncation=True, padding="max_length"),
        batched=True,
    )

    # Mapper la colonne 'difficulty' à 'labels'
    encoded_dataset = encoded_dataset.map(
        lambda e: {"labels": e["difficulty"]}, remove_columns=["difficulty", "sentence"]
    )
    # Convertir les listes en tensors
    encoded_dataset.set_format(
        "torch", columns=["input_ids", "attention_mask", "labels"]
    )

    # Split into train and test
    split_dataset = encoded_dataset.train_test_split(test_size=0.2)
    train_dataset = split_dataset["train"]
    test_dataset = split_dataset["test"]

    return train_dataset, test_dataset, label_encoder


ljl_train_dataset, ljl_test_dataset, label_encoder = prepare_bert_datasets(
    csv_list["ljl"]["train"], AutoTokenizer.from_pretrained("camembert-base")
)
len(ljl_train_dataset)

In [ ]:
# ----------------------------- SLURMRAY TRAINING ---------------------------- #
from transformers import (
    CamembertForSequenceClassification,
    Trainer,
    TrainingArguments,
    EarlyStoppingCallback,
)
from transformers import AutoTokenizer
import pandas as pd
import os
import ray.train.huggingface
import ray.train.huggingface.transformers
import dill


def train_camembert(
    df: pd.DataFrame,
    model_name: str = "camembert-base",
    epochs: int = 3,
    use_ray: bool = False,
):
    # Fix partial import bug
    import ray.train.huggingface
    import ray.train.huggingface.transformers

    # Charger le tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    # Préparer les datasets
    train_dataset, test_dataset, label_encoder = prepare_bert_datasets(df, tokenizer)

    # Charger le modèle et le tokenizer
    model = CamembertForSequenceClassification.from_pretrained(
        model_name, num_labels=len(df["difficulty"].unique()), return_dict=True
    )

    # Create model folder if it doesn't exist
    path = os.path.join("models", "difficulty_estimation", model_name.replace("/", "_"))
    if not os.path.exists(path):
        os.makedirs(path)

    # Créer le Trainer
    training_args = TrainingArguments(
        output_dir=path,  # répertoire de sortie pour les fichiers de modèle
        num_train_epochs=epochs,  # nombre d'époques
        per_device_train_batch_size=32,  # taille du lot d'entraînement
        per_device_eval_batch_size=64,  # taille du lot d'évaluation
        warmup_steps=500,  # nombre d'étapes de réchauffement à effectuer avant de commencer à décroître le taux d'apprentissage
        weight_decay=0.01,  # taux de décroissance du poids
        logging_dir="./.logs",  # répertoire pour les journaux
        evaluation_strategy="steps",
        eval_steps=50,
        load_best_model_at_end=True,
        metric_for_best_model="eval_loss",
        greater_is_better=False,
        save_strategy="steps",  # Change this line
        save_steps=50,  # And this line
        save_total_limit=1,  # And add this line
    )

    early_stopping = EarlyStoppingCallback(
        early_stopping_patience=3
    )  # Définir le nombre de validation steps avant l'arrêt

    trainer = Trainer(
        model=model,  # le modèle à entraîner
        args=training_args,  # les arguments d'entraînement
        train_dataset=train_dataset,  # le jeu de données d'entraînement
        eval_dataset=test_dataset,  # le jeu de données d'évaluation
        callbacks=[early_stopping],
    )

    if use_ray:
        # Add ray tune callback
        callback = ray.train.huggingface.transformers.RayTrainReportCallback()
        trainer.add_callback(callback)
        trainer = ray.train.huggingface.transformers.prepare_trainer(trainer)

    # Entraîner le modèle
    trainer.train()

    # Pickle result
    with open(os.path.join("train_camembert_result.pkl"), "wb") as f:
        dill.dump((tokenizer, label_encoder), f)

    return trainer, model, tokenizer, label_encoder

In [ ]:
# ------------------------- DEFINE SLURMRAY FUNCTION ------------------------- #
from slurmray.RayLauncher import RayLauncher
from ray.train import ScalingConfig, CheckpointConfig, RunConfig
from ray.train.torch import TorchTrainer
import shutil

MODEL_NAME = "camembert-base"


def ray_launcher(config):
    # Get config
    kwargs = config["kwargs"]
    f = config["f"]

    # Run function
    return f(**kwargs)


def slurmray_function(df: pd.DataFrame):
    # Create ray launcher
    ray_trainer = TorchTrainer(
        ray_launcher,
        scaling_config=ScalingConfig(num_workers=1, use_gpu=True),
        run_config=RunConfig(checkpoint_config=CheckpointConfig(num_to_keep=1)),
        train_loop_config={
            "f": train_camembert,
            "kwargs": {
                "df": df,
                "model_name": MODEL_NAME,
                "epochs": 10,
                "use_ray": True,
            },
        },
    )

    # Start training
    result = ray_trainer.fit()

    # Retreive result
    with open(os.path.join(result.path, "train_camembert_result.pkl"), "rb") as f:
        tokenizer, label_encoder = dill.load(f)

    # Find only checkpoint folder
    checkpoint_path = os.path.join(
        result.path, "models", "difficulty_estimation", MODEL_NAME
    )
    for dirpath, dirnames, filenames in os.walk(checkpoint_path):
        for dirname in dirnames:
            checkpoint_path = os.path.join(dirpath, dirname)

    # Zip checkpoint folder
    shutil.make_archive(checkpoint_path, "zip", checkpoint_path)

    # Load checkpoint and return it
    checkpoint_path = checkpoint_path + ".zip"
    with open(os.path.join(checkpoint_path), "rb") as f:
        file_content = f.read()

    return result, tokenizer, label_encoder, file_content


bert_launcher_builder = lambda dataset: RayLauncher(
    project_name="camembert_base_difficulty_estimation",
    func=slurmray_function,
    args={"df": csv_list[dataset]["train"]},
    modules=[],
    node_nbr=2,
    use_gpu=True,
    memory=128,
    max_running_time=60,
    server_run=True,
    server_ssh="curnagl.dcsr.unil.ch",
    server_username="hjamet",
)

In [ ]:
# ---------------------------- TRAIN CAMEMBERT... ---------------------------- #


def train_camembert_local(dataset: str):
    launcher = bert_launcher_builder(dataset)
    trainer_result, tokenizer, label_encoder, checkpoint = launcher()
    # Unzip predictions
    file_name = os.path.join(
        "results", "OpenSourceModelTraining", f"Bert_{dataset}.zip"
    )
    # Create folder if it doesn't exist
    if not os.path.exists(os.path.dirname(file_name)):
        os.makedirs(os.path.dirname(file_name))
    with open(file_name, "wb") as f:
        f.write(checkpoint)
    shutil.unpack_archive(file_name, file_name.replace(".zip", ""))
    os.remove(file_name)

    # write tokenizer and label encoder
    with open(
        os.path.join(
            file_name.replace(".zip", ""), "train_camembert_tokenizer_label_encoder.pkl"
        ),
        "wb",
    ) as f:
        dill.dump((tokenizer, label_encoder), f)

    return trainer_result

In [7]:
# --------------------------------- ...ON LJL -------------------------------- #
train_camembert_local("ljl")

Serializing function and arguments...
Connecting to the cluster...


INFO:paramiko.transport:Connected (version 2.0, client OpenSSH_8.0)
INFO:paramiko.transport:Authentication (password) successful!
INFO:paramiko.transport.sftp:[chan 0] Opened sftp connection (server version 3)


Writing slurmray server script...
Running server...
Installing slurmray server
Writing python script...
Writing slurm script...
No serialization done.
Cluster detected, running on cluster...
Canceling old jobs...
Start to submit job!
Job submitted! Script file is at: </users/hjamet/slurmray-server/.slogs/server/sbatch.sh>. Log file is at: </users/hjamet/slurmray-server/.slogs/server/server_1212-16h29.log>
Start to monitor the queue... You can check the queue at: </users/hjamet/slurmray-server/.slogs/server/server_1212-16h29_queue.log>
Submitted batch job 35878856
IP Head: 10.203.101.83:6379
STARTING HEAD at dnagpu003
2023-12-12 16:29:03,994	INFO usage_lib.py:416 -- Usage stats collection is enabled by default without user confirmation because this terminal is detected to be non-interactive. To disable this, add `--disable-usage-stats` to the command that starts the cluster, or run the following command: `ray disable-usage-stats` before starting the cluster. See https://docs.ray.io/en/m

Result(
  metrics={'eval_loss': 1.0293720960617065, 'eval_runtime': 2.5031, 'eval_samples_per_second': 131.839, 'eval_steps_per_second': 2.397, 'epoch': 9.52, 'step': 400},
  path='/users/hjamet/ray_results/TorchTrainer_2023-12-12_16-30-18/TorchTrainer_5ae89_00000_0_2023-12-12_16-30-18',
  filesystem='local',
  checkpoint=Checkpoint(filesystem=local, path=/users/hjamet/ray_results/TorchTrainer_2023-12-12_16-30-18/TorchTrainer_5ae89_00000_0_2023-12-12_16-30-18/checkpoint_000007)
)

In [8]:
# -------------------------- ...ON FRENCH_DIFFICULTY ------------------------- #
train_camembert_local("french_difficulty")

Serializing function and arguments...
Connecting to the cluster...


INFO:paramiko.transport:Connected (version 2.0, client OpenSSH_8.0)
INFO:paramiko.transport:Authentication (password) successful!
INFO:paramiko.transport.sftp:[chan 0] Opened sftp connection (server version 3)


Writing slurmray server script...
Running server...
Installing slurmray server
Writing python script...
Writing slurm script...
No serialization done.
Cluster detected, running on cluster...
Canceling old jobs...
Start to submit job!
Job submitted! Script file is at: </users/hjamet/slurmray-server/.slogs/server/sbatch.sh>. Log file is at: </users/hjamet/slurmray-server/.slogs/server/server_1212-16h49.log>
Start to monitor the queue... You can check the queue at: </users/hjamet/slurmray-server/.slogs/server/server_1212-16h49_queue.log>
Submitted batch job 35879437
IP Head: 10.203.101.82:6379
STARTING HEAD at dnagpu002
2023-12-12 16:49:28,093	INFO usage_lib.py:416 -- Usage stats collection is enabled by default without user confirmation because this terminal is detected to be non-interactive. To disable this, add `--disable-usage-stats` to the command that starts the cluster, or run the following command: `ray disable-usage-stats` before starting the cluster. See https://docs.ray.io/en/m

Result(
  metrics={'eval_loss': 1.2243598699569702, 'eval_runtime': 5.375, 'eval_samples_per_second': 142.883, 'eval_steps_per_second': 2.233, 'epoch': 5.73, 'step': 550, 'loss': 1.2882, 'learning_rate': 5e-05},
  path='/users/hjamet/ray_results/TorchTrainer_2023-12-12_16-50-44/TorchTrainer_358f4_00000_0_2023-12-12_16-50-44',
  filesystem='local',
  checkpoint=Checkpoint(filesystem=local, path=/users/hjamet/ray_results/TorchTrainer_2023-12-12_16-50-44/TorchTrainer_358f4_00000_0_2023-12-12_16-50-44/checkpoint_000010)
)

In [9]:
# ------------------------------ ...ON SENTENCES ----------------------------- #
train_camembert_local("sentences")

Serializing function and arguments...
Connecting to the cluster...


INFO:paramiko.transport:Connected (version 2.0, client OpenSSH_8.0)
INFO:paramiko.transport:Authentication (password) successful!
INFO:paramiko.transport.sftp:[chan 0] Opened sftp connection (server version 3)


Writing slurmray server script...
Running server...
Installing slurmray server
Writing python script...
Writing slurm script...
No serialization done.
Cluster detected, running on cluster...
Canceling old jobs...
Start to submit job!
Job submitted! Script file is at: </users/hjamet/slurmray-server/.slogs/server/sbatch.sh>. Log file is at: </users/hjamet/slurmray-server/.slogs/server/server_1212-17h10.log>
Start to monitor the queue... You can check the queue at: </users/hjamet/slurmray-server/.slogs/server/server_1212-17h10_queue.log>
Submitted batch job 35879773
IP Head: 10.203.101.87:6379
STARTING HEAD at dnagpu007
2023-12-12 17:10:27,153	INFO usage_lib.py:416 -- Usage stats collection is enabled by default without user confirmation because this terminal is detected to be non-interactive. To disable this, add `--disable-usage-stats` to the command that starts the cluster, or run the following command: `ray disable-usage-stats` before starting the cluster. See https://docs.ray.io/en/m

Result(
  metrics={'eval_loss': 0.4600411355495453, 'eval_runtime': 2.6877, 'eval_samples_per_second': 142.873, 'eval_steps_per_second': 2.232, 'epoch': 9.38, 'step': 450},
  path='/users/hjamet/ray_results/TorchTrainer_2023-12-12_17-11-41/TorchTrainer_22e01_00000_0_2023-12-12_17-11-41',
  filesystem='local',
  checkpoint=Checkpoint(filesystem=local, path=/users/hjamet/ray_results/TorchTrainer_2023-12-12_17-11-41/TorchTrainer_22e01_00000_0_2023-12-12_17-11-41/checkpoint_000008)
)

## Mistral-7B

In [5]:
# ------------------------------ PREPARE DATASET ----------------------------- #
from transformers import AutoTokenizer
from datasets import Dataset
import pandas as pd
from sklearn.preprocessing import LabelEncoder


def prepare_mistral_datasets(
    df: pd.DataFrame,
    tokenizer: AutoTokenizer,
    context: str = None,
    split_size: float = 0.2,
):
    # Encode labels
    label_encoder = LabelEncoder()
    df["difficulty"] = label_encoder.fit_transform(df["difficulty"])
    df.astype({"difficulty": "str"})

    # Create dataset
    dataset = Dataset.from_pandas(df)

    # Remove labels if split_size is 0 (for inference)
    if split_size == 0:
        dataset = dataset.map(
            lambda example: {"sentence": example["sentence"], "difficulty": ""},
            remove_columns=["difficulty"],
        )

    # Add context, label and tokenize
    max_length = 512

    def simple_tokenize(e):
        result = tokenizer(
            e["sentence"],
        )
        return result

    encoded_sentences = dataset.map(simple_tokenize)
    ## Truncate sentences
    truncated_dataset = encoded_sentences.map(
        lambda e: {
            "sentence": tokenizer.decode(e["input_ids"][: int(max_length)]),
        }
    ).remove_columns(["input_ids", "attention_mask"])

    # Determine Context size
    full_context_string = f"[INST] <<SYS>>\n{context if context is not None else ''}\n<</SYS>>\n\n [/INST] {truncated_dataset['difficulty'][0]}"
    context_length = len(tokenizer(full_context_string)["input_ids"])
    max_length = int(max_length + context_length) + 2

    def tokenize(e):
        context_str = (
            f"[INST] <<SYS>>\n{context if context is not None else ''}\n<</SYS>>\n\n"
        )
        formated = f"{context_str}{e['sentence']} [/INST] {e['difficulty']}"
        result = tokenizer(
            formated, padding="max_length", truncation=True, max_length=max_length
        )
        result["labels"] = result["input_ids"].copy()
        return result

    encoded_dataset = truncated_dataset.map(tokenize)

    # Convertir les listes en tensors
    encoded_dataset.set_format(
        "torch", columns=["input_ids", "attention_mask", "labels"]
    )

    # Remove unnecessary columns
    encoded_dataset = encoded_dataset.remove_columns(["sentence", "difficulty"])

    # Split into train and test
    if split_size == 0:
        return encoded_dataset
    split_dataset = encoded_dataset.train_test_split(test_size=split_size)
    train_dataset = split_dataset["train"]
    test_dataset = split_dataset["test"]

    return train_dataset, test_dataset


tokenizer = AutoTokenizer.from_pretrained(
    "bofenghuang/vigostral-7b-chat",
    padding_side="left",
    add_eos_token=True,
    add_bos_token=True,
    truncation_side="left",
)
tokenizer.pad_token = tokenizer.eos_token
ljl_train_dataset, ljl_test_dataset = prepare_mistral_datasets(
    csv_list["ljl"]["train"], tokenizer=tokenizer, split_size=0.2
)
print(len(ljl_test_dataset))

Map:   0%|          | 0/1647 [00:00<?, ? examples/s]

Map:   0%|          | 0/1647 [00:00<?, ? examples/s]

Map:   0%|          | 0/1647 [00:00<?, ? examples/s]

330


In [6]:
# Decode dataset
display(
    pd.Series(
        ljl_test_dataset.map(
            lambda e: {"sentence": tokenizer.decode(e["input_ids"])},
            remove_columns=["input_ids", "attention_mask", "labels"],
        )["sentence"]
    )
    .apply(lambda x: x[:20])
    .value_counts()
)

pd.Series(
    ljl_train_dataset.map(
        lambda e: {"size": len(e["input_ids"])},
        remove_columns=["input_ids", "attention_mask", "labels"],
    )["size"]
).astype(int).describe()

Map:   0%|          | 0/330 [00:00<?, ? examples/s]

</s></s></s></s></s>      290
</s><s> [INST] <<SYS       39
<s> [INST] <<SYS>>\n\n      1
Name: count, dtype: int64

Map:   0%|          | 0/1317 [00:00<?, ? examples/s]

count    1317.0
mean      536.0
std         0.0
min       536.0
25%       536.0
50%       536.0
75%       536.0
max       536.0
dtype: float64

In [7]:
# ----------------------------- SLURMRAY TRAINING ---------------------------- #
from transformers import (
    AutoModelForCausalLM,
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling,
    EarlyStoppingCallback,
)
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model
from transformers import AutoTokenizer
import pandas as pd
import os
import ray.train.huggingface
import ray.train.huggingface.transformers
import dill
import torch
from datetime import datetime


def train_mistral(
    train_set: pd.DataFrame,
    model_name: str = "mistralai/Mistral-7B-v0.1",
    context: str = None,
    use_ray: bool = False,
    pwd: str = ".",
    max_steps: int = 325,
):
    # Fix partial import bug
    import ray.train.huggingface
    import ray.train.huggingface.transformers

    # Charger le tokenizer
    tokenizer = AutoTokenizer.from_pretrained(
        model_name,
        padding_side="left",
        truncation_side="left",
        add_eos_token=True,
        add_bos_token=True,
    )
    tokenizer.pad_token = tokenizer.eos_token

    # Préparer les datasets
    train_dataset, validation_dataset = prepare_mistral_datasets(
        train_set, tokenizer, context=context, split_size=0.2
    )

    # Charger le modèle et le tokenizer
    # nf4_config = BitsAndBytesConfig(
    #     load_in_4bit=True,
    #     bnb_4bit_quant_type="nf4",
    #     bnb_4bit_use_double_quant=True,
    #     bnb_4bit_compute_dtype=torch.bfloat16,
    # )
    model = AutoModelForCausalLM.from_pretrained(
        model_name, device_map="auto", use_cache=False
    )

    # Configure model
    config = LoraConfig(
        r=64,  # Plus r est grand, plus le modèle est précis mais plus il est lent
        lora_alpha=16,
        target_modules=[
            "q_proj",
            "k_proj",
            "v_proj",
            "o_proj",
            "gate_proj",
            "up_proj",
            "down_proj",
            "lm_head",
        ],
        bias="none",
        lora_dropout=0.05,
        task_type="CAUSAL_LM",
    )

    model.gradient_checkpointing_enable()
    model = prepare_model_for_kbit_training(model)
    model = get_peft_model(model, config)
    model.config.use_cache = False

    # Create model folder if it doesn't exist
    path = os.path.join(
        pwd, "models", "difficulty_estimation", model_name.replace("/", "_")
    )
    if not os.path.exists(path):
        os.makedirs(path)

    # DeepSpeed config
    # deepspeed = {
    #     "fp16": {
    #         "enabled": "auto",
    #         "initial_scale_power": 8,
    #     },
    #     "bf16": {"enabled": "auto"},
    #     "optimizer": {
    #         "type": "AdamW",
    #         "params": {
    #             "lr": "auto",
    #             "betas": "auto",
    #             "eps": "auto",
    #         },
    #     },
    #     "zero_optimization": {
    #         "stage": 3,
    #         "offload_optimizer": {
    #             "device": "cpu",
    #             "pin_memory": True,
    #         },
    #         "offload_param": {
    #             "device": "cpu",
    #             "pin_memory": True,
    #         },
    #         "overlap_comm": True,
    #         "contiguous_gradients": True,
    #         "reduce_bucket_size": "auto",
    #         "stage3_prefetch_bucket_size": "auto",
    #         "stage3_param_persistence_threshold": "auto",
    #         "gather_16bit_weights_on_model_save": True,
    #         "round_robin_gradients": True,
    #     },
    #     "gradient_accumulation_steps": "auto",
    #     "gradient_clipping": "auto",
    #     "steps_per_print": 10,
    #     "train_batch_size": "auto",
    #     "train_micro_batch_size_per_gpu": "auto",
    #     "wall_clock_breakdown": False,
    # }

    # Configure WandB
    os.environ["WANDB_PROJECT"] = "mistral_difficulty_estimation"

    # Early stopping
    early_stopping = EarlyStoppingCallback(early_stopping_patience=3)

    # Créer le Trainer
    training_args = TrainingArguments(
        output_dir=path,
        warmup_steps=1,
        per_device_train_batch_size=8,
        gradient_accumulation_steps=1,
        max_steps=max_steps,
        learning_rate=2.5e-5,  # Want a small lr for finetuning
        bf16=True,
        optim="paged_adamw_8bit",
        logging_steps=25,  # When to start reporting loss
        logging_dir="./logs",  # Directory for storing logs
        save_strategy="steps",  # Save the model checkpoint every logging step
        save_steps=250,  # Save checkpoints every 500 steps
        evaluation_strategy="steps",  # Evaluate the model every logging step
        eval_steps=25,  # Evaluate and save checkpoints every 50 steps
        do_eval=True,  # Perform evaluation at the end of training
        ddp_find_unused_parameters=False,  # Necessary for FSDP to work
        # deepspeed=deepspeed,  # Much better GPU memory distribution
        # eval_accumulation_steps=1,  # To avoid OOM
        run_name=f"{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}",
        report_to="wandb",
        load_best_model_at_end=True,
    )

    trainer = Trainer(
        model=model,  # le modèle à entraîner
        args=training_args,  # les arguments d'entraînement
        train_dataset=train_dataset,  # le jeu de données d'entraînement
        eval_dataset=validation_dataset,  # le jeu de données d'évaluation
        data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),
        callbacks=[early_stopping],
    )

    if use_ray:
        # Add ray tune callback
        callback = ray.train.huggingface.transformers.RayTrainReportCallback()
        trainer.add_callback(callback)
        trainer = ray.train.huggingface.transformers.prepare_trainer(trainer)

    # Entraîner le modèle
    result = trainer.train()

    # Write checkpoint
    trainer.save_model(os.path.join(pwd, "mistral_trained"))

In [8]:
# ------------------------- DEFINE SLURMRAY FUNCTION ------------------------- #
from slurmray.RayLauncher import RayLauncher
from ray.train import ScalingConfig, CheckpointConfig, RunConfig, FailureConfig
from ray.train.torch import TorchTrainer

MODEL_NAME = "bofenghuang/vigostral-7b-chat"


def ray_launcher(config):
    # Get config
    kwargs = config["kwargs"]
    f = config["f"]

    # Run function
    return f(**kwargs)


def slurmray_function(
    train_df: pd.DataFrame, context: str = None, max_steps: int = 350
):
    # Create ray launcher
    ray_trainer = TorchTrainer(
        ray_launcher,
        scaling_config=ScalingConfig(num_workers=1, use_gpu=True),
        run_config=RunConfig(
            checkpoint_config=CheckpointConfig(
                num_to_keep=1
            ),  # No more needed with the new storage path
            storage_path="/scratch/hjamet",
        ),
        train_loop_config={
            "f": train_mistral,
            "kwargs": {
                "train_set": train_df,
                "model_name": MODEL_NAME,
                "context": context,
                "use_ray": True,
                "pwd": "/scratch/hjamet",
                "max_steps": max_steps,
            },
        },
    )

    # Start training
    result = ray_trainer.fit()

    # Retreive result
    return result


mistral_launcher_builder = lambda dataset, context: RayLauncher(
    project_name="mistral_difficulty_estimation",
    func=slurmray_function,
    args={
        "train_df": csv_list[dataset]["train"],
        "context": context,
        "max_steps": 1000,  # Minimum of the learning curve
    },
    modules=[],
    node_nbr=1,
    use_gpu=True,
    memory=128,
    max_running_time=60 * 2,
    server_run=True,
    server_ssh="curnagl.dcsr.unil.ch",
    server_username="hjamet",
)

In [9]:
# ----------------------------- TRAIN MISTRAL... ----------------------------- #
def train_mistral_local(dataset: str, context: str):
    launcher = mistral_launcher_builder(dataset, context)
    predictions = launcher()

    return predictions

In [25]:
# --------------------------------- ...ON LJL -------------------------------- #
result = train_mistral_local("ljl", context="")

Serializing function and arguments...
Connecting to the cluster...


INFO:paramiko.transport:Connected (version 2.0, client OpenSSH_8.0)
INFO:paramiko.transport:Authentication (password) successful!
INFO:paramiko.transport.sftp:[chan 0] Opened sftp connection (server version 3)


Writing slurmray server script...
Running server...
Installing slurmray server
Looking in indexes: https://download.pytorch.org/whl/cu121
Writing python script...
Writing slurm script...
No serialization done.
Cluster detected, running on cluster...
Canceling old jobs...
Start to submit job!
Job submitted! Script file is at: </users/hjamet/slurmray-server/.slogs/server/sbatch.sh>. Log file is at: </users/hjamet/slurmray-server/.slogs/server/server_1801-18h50.log>
Start to monitor the queue... You can check the queue at: </users/hjamet/slurmray-server/.slogs/server/server_1801-18h50_queue.log>
Submitted batch job 37140719
IP Head: 10.203.101.82:6379
STARTING HEAD at dnagpu002
2024-01-18 18:50:49,070	INFO usage_lib.py:416 -- Usage stats collection is enabled by default without user confirmation because this terminal is detected to be non-interactive. To disable this, add `--disable-usage-stats` to the command that starts the cluster, or run the following command: `ray disable-usage-stats

In [26]:
result = train_mistral_local(
    "ljl",
    context="Vous êtes un évaluateur linguistique utilisant le Cadre européen commun de référence pour les langues (CECRL). Votre mission est d'attribuer une note de compétence linguistique à ce texte, en utilisant les niveaux du CECRL, allant de A1 (débutant) à C2 (avancé/natif). Évaluez ce texte et attribuez-lui la note correspondante du CECRL.",
)

Serializing function and arguments...
Connecting to the cluster...


INFO:paramiko.transport:Connected (version 2.0, client OpenSSH_8.0)
INFO:paramiko.transport:Authentication (password) successful!
INFO:paramiko.transport.sftp:[chan 0] Opened sftp connection (server version 3)


Writing slurmray server script...
Running server...
Installing slurmray server
Looking in indexes: https://download.pytorch.org/whl/cu121
Writing python script...
Writing slurm script...
No serialization done.
Cluster detected, running on cluster...
Canceling old jobs...
Start to submit job!
Job submitted! Script file is at: </users/hjamet/slurmray-server/.slogs/server/sbatch.sh>. Log file is at: </users/hjamet/slurmray-server/.slogs/server/server_1801-20h06.log>
Start to monitor the queue... You can check the queue at: </users/hjamet/slurmray-server/.slogs/server/server_1801-20h06_queue.log>
Submitted batch job 37142183
IP Head: 10.203.101.82:6379
STARTING HEAD at dnagpu002
2024-01-18 20:06:47,896	INFO usage_lib.py:416 -- Usage stats collection is enabled by default without user confirmation because this terminal is detected to be non-interactive. To disable this, add `--disable-usage-stats` to the command that starts the cluster, or run the following command: `ray disable-usage-stats

In [8]:
# ------------------------------- ON SENTENCES ------------------------------- #
result = train_mistral_local(
    "sentences",
    context="",
)

Serializing function and arguments...
Connecting to the cluster...


INFO:paramiko.transport:Connected (version 2.0, client OpenSSH_8.0)
INFO:paramiko.transport:Authentication (password) successful!
INFO:paramiko.transport.sftp:[chan 0] Opened sftp connection (server version 3)


Writing slurmray server script...
Running server...
Installing slurmray server
Looking in indexes: https://download.pytorch.org/whl/cu121
Writing python script...
Writing slurm script...
No serialization done.
Cluster detected, running on cluster...
Canceling old jobs...
Start to submit job!
Job submitted! Script file is at: </users/hjamet/slurmray-server/.slogs/server/sbatch.sh>. Log file is at: </users/hjamet/slurmray-server/.slogs/server/server_1801-23h06.log>
Start to monitor the queue... You can check the queue at: </users/hjamet/slurmray-server/.slogs/server/server_1801-23h06_queue.log>
Submitted batch job 37144029
IP Head: 10.203.101.82:6379
STARTING HEAD at dnagpu002
2024-01-18 23:06:28,346	INFO usage_lib.py:416 -- Usage stats collection is enabled by default without user confirmation because this terminal is detected to be non-interactive. To disable this, add `--disable-usage-stats` to the command that starts the cluster, or run the following command: `ray disable-usage-stats

In [11]:
# ------------------------------- ON SENTENCES ------------------------------- #
result = train_mistral_local(
    "sentences",
    context="Vous êtes un évaluateur linguistique utilisant le Cadre européen commun de référence pour les langues (CECRL). Votre mission est d'attribuer une note de compétence linguistique à ce texte, en utilisant les niveaux du CECRL, allant de A1 (débutant) à C2 (avancé/natif). Évaluez ce texte et attribuez-lui la note correspondante du CECRL.",
)

Serializing function and arguments...
Connecting to the cluster...


INFO:paramiko.transport:Connected (version 2.0, client OpenSSH_8.0)
INFO:paramiko.transport:Authentication (password) successful!
INFO:paramiko.transport.sftp:[chan 0] Opened sftp connection (server version 3)


Writing slurmray server script...
Running server...
Installing slurmray server
Looking in indexes: https://download.pytorch.org/whl/cu121
Writing python script...
Writing slurm script...
No serialization done.
Cluster detected, running on cluster...
Canceling old jobs...
Start to submit job!
Job submitted! Script file is at: </users/hjamet/slurmray-server/.slogs/server/sbatch.sh>. Log file is at: </users/hjamet/slurmray-server/.slogs/server/server_1901-10h48.log>
Start to monitor the queue... You can check the queue at: </users/hjamet/slurmray-server/.slogs/server/server_1901-10h48_queue.log>
Submitted batch job 37152560
IP Head: 10.203.101.88:6379
STARTING HEAD at dnagpu008
2024-01-19 10:52:02,085	INFO usage_lib.py:416 -- Usage stats collection is enabled by default without user confirmation because this terminal is detected to be non-interactive. To disable this, add `--disable-usage-stats` to the command that starts the cluster, or run the following command: `ray disable-usage-stats

In [12]:
# ------------------------------- ON SENTENCES ------------------------------- #
result = train_mistral_local(
    "french_difficulty",
    context="",
)

Serializing function and arguments...
Connecting to the cluster...


INFO:paramiko.transport:Connected (version 2.0, client OpenSSH_8.0)
INFO:paramiko.transport:Authentication (password) successful!
INFO:paramiko.transport.sftp:[chan 0] Opened sftp connection (server version 3)


Writing slurmray server script...
Running server...
Installing slurmray server
Looking in indexes: https://download.pytorch.org/whl/cu121
Writing python script...
Writing slurm script...
No serialization done.
Cluster detected, running on cluster...
Canceling old jobs...
Start to submit job!
Job submitted! Script file is at: </users/hjamet/slurmray-server/.slogs/server/sbatch.sh>. Log file is at: </users/hjamet/slurmray-server/.slogs/server/server_1901-12h20.log>
Start to monitor the queue... You can check the queue at: </users/hjamet/slurmray-server/.slogs/server/server_1901-12h20_queue.log>
Submitted batch job 37153506
IP Head: 10.203.101.88:6379
STARTING HEAD at dnagpu008
2024-01-19 12:26:48,677	INFO usage_lib.py:416 -- Usage stats collection is enabled by default without user confirmation because this terminal is detected to be non-interactive. To disable this, add `--disable-usage-stats` to the command that starts the cluster, or run the following command: `ray disable-usage-stats

In [13]:
# ------------------------------- ON SENTENCES ------------------------------- #
result = train_mistral_local(
    "french_difficulty",
    context="Vous êtes un évaluateur linguistique utilisant le Cadre européen commun de référence pour les langues (CECRL). Votre mission est d'attribuer une note de compétence linguistique à ce texte, en utilisant les niveaux du CECRL, allant de A1 (débutant) à C2 (avancé/natif). Évaluez ce texte et attribuez-lui la note correspondante du CECRL.",
)

Serializing function and arguments...
Connecting to the cluster...


INFO:paramiko.transport:Connected (version 2.0, client OpenSSH_8.0)
INFO:paramiko.transport:Authentication (password) successful!
INFO:paramiko.transport.sftp:[chan 0] Opened sftp connection (server version 3)


Writing slurmray server script...
Running server...
Installing slurmray server
Looking in indexes: https://download.pytorch.org/whl/cu121
Writing python script...
Writing slurm script...
No serialization done.
Cluster detected, running on cluster...
Canceling old jobs...
Start to submit job!
Job submitted! Script file is at: </users/hjamet/slurmray-server/.slogs/server/sbatch.sh>. Log file is at: </users/hjamet/slurmray-server/.slogs/server/server_1901-14h07.log>
Start to monitor the queue... You can check the queue at: </users/hjamet/slurmray-server/.slogs/server/server_1901-14h07_queue.log>
Submitted batch job 37155125
IP Head: 10.203.101.88:6379
STARTING HEAD at dnagpu008
2024-01-19 14:07:18,982	INFO usage_lib.py:416 -- Usage stats collection is enabled by default without user confirmation because this terminal is detected to be non-interactive. To disable this, add `--disable-usage-stats` to the command that starts the cluster, or run the following command: `ray disable-usage-stats